In [7]:
METABASE_SSH = None

In [8]:
f = FILE_COMMON_PATH + "lib_common.ipynb"
%run $f

f = FILE_COMMON_PATH + "lib_db.ipynb"
%run $f

In [ ]:
f = FILE_COMMON_PATH + "config.ipynb"
%run $f

In [ ]:
f = FILE_COMMON_PATH + "lib_ml.ipynb"
%run $f

In [15]:
AZURE_DB['schema'] = 'dds'

DB_WRITE = AZURE_DB.copy()
TABLE_WRITE = 'recommendation'
DB_WRITE['schema'] = 'dds'
DB_WRITE['table'] = TABLE_WRITE

DB_TEMP = DB_WRITE.copy()
TABLE_TEMP = TABLE_WRITE + "_tmp"
DB_TEMP['schema'] = 'history'
DB_TEMP['table'] = TABLE_TEMP

{'drivername': 'mssql+pyodbc',
 'username': 'administrator2019',
 'password': 'paysmart@2019',
 'host': 'paysmart-svr.database.windows.net',
 'port': 1433,
 'database': 'paysmart_dwh',
 'driver': 'ODBC Driver 18 for SQL Server'}

In [12]:
def get_trans_group(spark_ss, group_field_1, group_field_2, calculate_field, calculate_func, from_date):
    
    hdfs_trans = "hdfs://master:9000/all_data/raw/az_transaction.parquet" 
    sdf_txn = spark_ss.read.parquet(hdfs_trans) 
    sdf_txn = sdf_txn.filter(sdf_txn.txn_status == 1)
    sdf_txn = sdf_txn.filter(sdf_txn.txn_date > from_date)
    
    if calculate_func == 'count':
        dft = sdf_txn.groupBy(group_field_1, group_field_2).count()
        return dft.withColumnRenamed(group_field_2, 'group_field').withColumnRenamed('count', 'cal_field')
    
    elif calculate_func == 'sum':
        dft = sdf_txn.groupBy(group_field_1, group_field_2).sum(calculate_field)
        return dft.withColumnRenamed(group_field_2, 'group_field').withColumnRenamed('sum', 'cal_field')
    
    return dft

In [13]:
import threading
from threading import Thread

class ThreadWithReturnValue(Thread):
    
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return


In [ ]:
from datetime import datetime, timedelta

running_date = datetime.now().strftime("%Y-%m-%d")
trans_from_date = (datetime.now() - timedelta(days=14)).strftime("%Y-%m-%d")
file_sim = "hdfs://master:9000/all_data/features/recommend_similarity.parquet"
file_sim_history = "hdfs://master:9000/all_data/features/recommend_sim_hist.parquet"
running_date, trans_from_date, file_sim, file_sim_history

In [ ]:
# Running all
from datetime import datetime, timedelta
import pyspark

conf = pyspark.SparkConf().setAll([('spark.master', 'spark://master:7077'),
                                    ('spark.executor.memory', '2g'), 
                                  ('spark.executor.cores', '2'), 
                                  ('spark.cores.max', '2'), 
                                  ('spark.driver.memory','10g')])

# This is to fix "writing dates before 1582-10-15 or timestamps before into Parquet INT96"
conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
conf.set('spark.driver.extraClassPath', '/usr/local/hadoop/share/hadoop/yarn/lib/mssql-jdbc-6.2.1.jre7.jar')
conf.set('spark.executor.extraClassPath', '/usr/local/hadoop/share/hadoop/yarn/lib/mssql-jdbc-6.2.1.jre7.jar')

sc = pyspark.SparkContext(conf=conf)

In [11]:
from pyspark.sql import SparkSession
import pyspark

spark_ss = SparkSession.builder.appName('recommendation_similarity').getOrCreate()

In [17]:
ls_group_field_2 = ['txn_type', 'service_code', 'bank_code', 'segment_1', 'segment_2', 'order_service', 
                'service_provider_code', 'order_info_partner_code', 'service_package_category', 
               'service_package_subcategory']
group_field_1 = "wallet_id"
calculate_field = "txn_amount"
calculate_func = "count"

In [ ]:
# Get all data
print('Get all data', datetime.now().strftime("%H:%M:%S"))
all_threads = [ThreadWithReturnValue(target=get_trans_group, 
                                args=(spark_ss, group_field_1, f2, calculate_field, calculate_func, trans_from_date)) 
    for f2 in ls_group_field_2]
[t.start() for t in all_threads]
ls_df = [t.join() for t in all_threads]

In [19]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import percent_rank, concat, lit
import pyspark.sql.functions as F

# Merge data
df_all, items_id, users_id = None, [], []
windowSpec  = Window.partitionBy("group_field")

for i, dft in enumerate(ls_df):
    dft = dft.filter(dft.group_field.isNotNull()).filter(dft.group_field != "None")\
        .withColumn("group_field", concat(lit(ls_group_field_2[i]), lit("_"), "group_field"))\
        .withColumn('maxi', F.max("cal_field").over(windowSpec))\
        .withColumn('mini', F.min("cal_field").over(windowSpec))\
        .withColumn("rating", F.round(99 * ((F.col("cal_field") - F.col('mini')) / (F.col('maxi') - F.col('mini'))) + 1, 2))\
        .drop('mini')\
        .drop('maxi')
    df_all = dft if df_all is None else df_all.union(dft)

In [ ]:
df_all = df_all.withColumn("user", df_all.wallet_id)
df_all = df_all.withColumn("item", df_all.group_field)
df_all.show()

In [ ]:
sc.setCheckpointDir("hdfs://master:9000/rdd_checkpoint")

In [ ]:
# Save similarity file
cf_rec = CF(df_all, 3, file_sim)
cf_rec.load_sim()
cf_rec.normalize_Y()

In [ ]:
# # Recommend all
# ps_new_rating = cf_rec.recommend_all(3)
# ps_new_rating.show(2)

In [ ]:
# Recommend full
ps_new_rating = cf_rec.recommend_all(None)

In [ ]:
def save_new_rating(ps_new_rating, file_recommend, METABASE_SSH, DB_WRITE, DB_TEMP):
    if ps_new_rating.count() > 0:
    
        # Save to HDFS
        print('Save to HDFS')
        ps_new_rating.write.mode("append").partitionBy('par_col').parquet(file_recommend)

        # Save to DB
        print('Save to DB')
        ps_new_rating.write.mode("overwrite") \
                .format("jdbc") \
                .option("url", f"jdbc:sqlserver://{DB_TEMP['host']};databaseName={DB_TEMP['database']}") \
                .option("dbtable", f"{DB_TEMP['schema']}.{DB_TEMP['table']}") \
                .option("user", DB_TEMP['username']) \
                .option("password", DB_TEMP['password']) \
                .save()
        upsert_sql(METABASE_SSH, DB_WRITE, DB_TEMP, {})    
    else:
        print('No existing record to write into DB / HDFS')

In [ ]:
from datetime import datetime

top_product = 5
top_user = 100

if top_product is not None:
    windowDept = Window.partitionBy("user_id").orderBy(F.col("predict_rating").desc())
    ps_new_rating_u = ps_new_rating.withColumn("top", \
                        F.row_number().over(windowDept)).filter(F.col("top") <= top_product) \
                        .select("user_id", "item_id", "predict_rating")\
                        .withColumn('par_col', F.lit(datetime.now().strftime("%Y-%m-%d")))        
    file_recommend = "hdfs://master:9000/all_data/features/recommendation.parquet"
    save_new_rating(ps_new_rating_u, file_recommend, METABASE_SSH, DB_WRITE, DB_TEMP)
    
    
    
if top_user is not None:
    windowDept = Window.partitionBy("item_id").orderBy(F.col("predict_rating").desc())
    ps_new_rating_i = ps_new_rating.withColumn("top", \
                        F.row_number().over(windowDept)).filter(F.col("top") <= top_user) \
                        .select("user_id", "item_id", "predict_rating")\
                        .withColumn('par_col', F.lit(datetime.now().strftime("%Y-%m-%d")))
    file_recommend = "hdfs://master:9000/all_data/features/recommendation_gb_item.parquet"
    DB_WRITE['table'] = 'recommendation_gb_item'
    DB_TEMP['table'] = DB_WRITE['table'] + "_tmp"
    save_new_rating(ps_new_rating_i, file_recommend, METABASE_SSH, DB_WRITE, DB_TEMP)

In [ ]:
sc.stop()